In [20]:
import pandas as pd
import ads
import requests
import PyPDF2
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import os
import numpy as np
import random
import re
import urllib3
import time
from requests.adapters import TimeoutSauce
import urllib, urllib.request    
from datetime import datetime
import feedparser
try:
    from urllib.request import urlopen
except ImportError:
    from urllib2 import urlopen
    

from urllib.parse import urlencode, quote_plus

## ADS

In [21]:
ADS_DEV_KEY = "FEbkvybqpxpJQdeF5E8btv7KvfFM4Eh5ezHV7Z8S"
ads.config.token = "FEbkvybqpxpJQdeF5E8btv7KvfFM4Eh5ezHV7Z8S"

In [ ]:
# https://github.com/adsabs/adsabs-dev-api/blob/master/API_documentation_Python/Search_API_Python.ipynb
# https://ui.adsabs.harvard.edu/help/search/comprehensive-solr-term-list
# https://solr.apache.org/guide/6_6/working-with-dates.html

# entdate: indicates when the record was first entered into our system
# update_timestamp: indicates when the record was updated in our system (small things such a new read/download will trigger an update)
# metadata_mtime: last time when the metadata (e.g., title, abstract) was modified
# nonbib_mtime: time when the other data (e.g., refereed, total citations) was modified
# fulltext_mtime: last time when the full text was modified
# orcid_mtime: last time when the ORCID claims were modified
# metrics_mtime: last time when metrics (e.g., historical reads, citations) were modified


In [23]:
start_month = "04"
start_year = "2023"

end_month = "04"
end_year = "2023"

for d in range(1, 30):
    start_day = d
    if int(start_day) < 10:
        start_day = f"0{start_day}"
    end_day = d + 1
    if int(end_day) < 10:
        end_day = f"0{end_day}"
    
    encoded_query = urlencode({"q": f"metadata_mtime:[{start_year}-{start_month}-{start_day}T00\:00\:00.000Z TO {end_year}-{end_month}-{end_day}T00\:00\:00.000Z]",
                           "fl": "bibcode, title, identifier",
                           "fq": "database:astronomy, property:article, property:(refereed OR eprint_openaccess), abs:(planet OR exoplanet OR extrasolar OR brown OR Jupiter OR Neptune OR TESS OR K2 OR Kepler OR TOI OR KOI OR OGLE OR KMT OR EPIC OR MOA)", 
                           "rows": 9999})
    
#     encoded_query = urlencode({"q": f"entdate:[{start_year}-{start_month}-{start_day} TO {end_year}-{end_month}-{end_day}]", 
#                            "fl": "bibcode, identifier, title", 
#                            "fq": "database:astronomy, property:article, property:(refereed OR eprint_openaccess), abs:(planet OR exoplanet OR extrasolar OR brown OR Jupiter OR Neptune OR TESS OR K2 OR Kepler OR TOI OR KOI OR OGLE OR KMT OR EPIC OR MOA)", 
#                            "rows": 9999})

    results = requests.get("https://api.adsabs.harvard.edu/v1/search/query?{}".format(encoded_query), headers={'Authorization': 'Bearer ' + ADS_DEV_KEY})
    ads_bibs = [paper['bibcode'] for paper in results.json()["response"]["docs"]]
    print(f"{start_day}-{end_day}: {len(ads_bibs)}")
    

# april
# metadata_mtime, "fq": "database:astronomy"
# 01-02: 8
# 02-03: 1802
# 03-04: 0
# 04-05: 0
# 05-06: 0
# 06-07: 38
# 07-08: 0
# 08-09: 0
# 09-10: 0
# 10-11: 0
# 11-12: 0
# 12-13: 0
# 13-14: 1419
# 14-15: 0
# 15-16: 143
# 16-17: 0
# 17-18: 0
# 18-19: 0
# 19-20: 0
# 20-21: 0
# 21-22: 0
# 22-23: 0
# 23-24: 0
# 24-25: 0
# 25-26: 0
# 26-27: 0
# 27-28: 0
# 28-29: 0
# 29-30: 0


# metadata_mtime, "fq": "database:astronomy, property:article, property:(refereed OR eprint_openaccess)"
# 01-02: 8
# 02-03: 1452
# 03-04: 0
# 04-05: 0
# 05-06: 0
# 06-07: 38
# 07-08: 0
# 08-09: 0
# 09-10: 0
# 10-11: 0
# 11-12: 0
# 12-13: 0
# 13-14: 1104
# 14-15: 0
# 15-16: 143
# 16-17: 0
# 17-18: 0
# 18-19: 0
# 19-20: 0
# 20-21: 0
# 21-22: 0
# 22-23: 0
# 23-24: 0
# 24-25: 0
# 25-26: 0
# 26-27: 0
# 27-28: 0
# 28-29: 0
# 29-30: 0
# 30-31: 55

# metadata_mtime, "fq": "database:astronomy, property:article, property:(refereed OR eprint_openaccess), abs:(planet OR exoplanet OR extrasolar OR brown OR Jupiter OR Neptune OR TESS OR K2 OR Kepler OR HD OR GJ or HIP OR TIC OR TOI OR KOI OR 2MASS OR OGLE OR KMT OR EPIC OR MOA -asteroid)"
# 01-02: 0
# 02-03: 264
# 03-04: 0
# 04-05: 0
# 05-06: 0
# 06-07: 2
# 07-08: 0
# 08-09: 0
# 09-10: 0
# 10-11: 0
# 11-12: 0
# 12-13: 0
# 13-14: 153
# 14-15: 0
# 15-16: 15
# 16-17: 0
# 17-18: 0
# 18-19: 0
# 19-20: 0
# 20-21: 0
# 21-22: 0
# 22-23: 0
# 23-24: 0
# 24-25: 0
# 25-26: 0
# 26-27: 0
# 27-28: 0
# 28-29: 0
# 29-30: 0

# metadata_mtime, "fq": "database:astronomy, property:article, property:(refereed OR eprint_openaccess), abs:(planet OR exoplanet OR extrasolar OR brown OR Jupiter OR Neptune OR TESS OR K2 OR Kepler OR TOI OR KOI OR OGLE OR KMT OR EPIC OR MOA)", 
# 01-02: 0
# 02-03: 306
# 03-04: 0
# 04-05: 0
# 05-06: 0
# 06-07: 2
# 07-08: 0
# 08-09: 0
# 09-10: 0
# 10-11: 0
# 11-12: 0
# 12-13: 0
# 13-14: 144
# 14-15: 0
# 15-16: 15
# 16-17: 0
# 17-18: 0
# 18-19: 0
# 19-20: 0
# 20-21: 0
# 21-22: 0
# 22-23: 0
# 23-24: 0
# 24-25: 0
# 25-26: 0
# 26-27: 0
# 27-28: 0
# 28-29: 0
# 29-30: 0



# entdate, "fq": "database:astronomy"
# 01-02: 1052
# 02-03: 55
# 03-04: 92
# 04-05: 116
# 05-06: 63
# 06-07: 46
# 07-08: 0
# 08-09: 794
# 09-10: 30
# 10-11: 66
# 11-12: 99
# 12-13: 51
# 13-14: 59
# 14-15: 0
# 15-16: 2000
# 16-17: 56
# 17-18: 108
# 18-19: 116
# 19-20: 52
# 20-21: 61
# 21-22: 0
# 22-23: 1346
# 23-24: 37
# 24-25: 74
# 25-26: 912
# 26-27: 70
# 27-28: 62
# 28-29: 0
# 29-30: 1127


# entdate, "fq": "database:astronomy, property:article, property:(refereed OR eprint_openaccess)"
# 01-02: 806
# 02-03: 55
# 03-04: 92
# 04-05: 116
# 05-06: 63
# 06-07: 46
# 07-08: 0
# 08-09: 451
# 09-10: 30
# 10-11: 66
# 11-12: 99
# 12-13: 51
# 13-14: 59
# 14-15: 0
# 15-16: 592
# 16-17: 56
# 17-18: 108
# 18-19: 116
# 19-20: 52
# 20-21: 61
# 21-22: 0
# 22-23: 522
# 23-24: 37
# 24-25: 74
# 25-26: 912
# 26-27: 70
# 27-28: 62
# 28-29: 0
# 29-30: 813


# entdate, "fq": "database:astronomy, property:article, property:(refereed OR eprint_openaccess), abs:(planet OR exoplanet OR extrasolar OR brown OR Jupiter OR Neptune OR TESS OR K2 OR Kepler OR HD OR GJ or HIP OR TIC OR TOI OR KOI OR 2MASS OR OGLE OR KMT OR EPIC OR MOA -asteroid)", 
# 01-02: 83
# 02-03: 9
# 03-04: 5
# 04-05: 8
# 05-06: 7
# 06-07: 8
# 07-08: 0
# 08-09: 66
# 09-10: 6
# 10-11: 8
# 11-12: 4
# 12-13: 5
# 13-14: 5
# 14-15: 0
# 15-16: 100
# 16-17: 4
# 17-18: 11
# 18-19: 9
# 19-20: 6
# 20-21: 4
# 21-22: 0
# 22-23: 79
# 23-24: 3
# 24-25: 6
# 25-26: 21
# 26-27: 8
# 27-28: 7
# 28-29: 0
# 29-30: 121

# entdate, "fq": "database:astronomy, property:article, property:(refereed OR eprint_openaccess), abs:(planet OR exoplanet OR extrasolar OR brown OR Jupiter OR Neptune OR TESS OR K2 OR Kepler OR TOI OR KOI OR OGLE OR KMT OR EPIC OR MOA)", 
# 01-02: 107
# 02-03: 10
# 03-04: 6
# 04-05: 6
# 05-06: 7
# 06-07: 9
# 07-08: 0
# 08-09: 69
# 09-10: 5
# 10-11: 8
# 11-12: 5
# 12-13: 6
# 13-14: 5
# 14-15: 0
# 15-16: 117
# 16-17: 4
# 17-18: 12
# 18-19: 9
# 19-20: 5
# 20-21: 4
# 21-22: 0
# 22-23: 85
# 23-24: 3
# 24-25: 6
# 25-26: 20
# 26-27: 8
# 27-28: 8
# 28-29: 0
# 29-30: 138


01-02: 0
02-03: 306
03-04: 0
04-05: 0
05-06: 0
06-07: 2
07-08: 0
08-09: 0
09-10: 0
10-11: 0
11-12: 0
12-13: 0
13-14: 144
14-15: 0
15-16: 15
16-17: 0
17-18: 0
18-19: 0
19-20: 0
20-21: 0
21-22: 0
22-23: 0
23-24: 0
24-25: 0
25-26: 0
26-27: 0
27-28: 0
28-29: 0
29-30: 0


In [20]:
encoded_query = urlencode({"q": "metadata_mtime:[2023-05-22T00\:00\:00.000Z TO 2023-05-23T00\:00\:00.000Z]",
                           "fl": "bibcode, title, identifier",
                           "fq": "database:astronomy",
                           "rows": 9999})

# entdate:[2023-04-18 TO 2023-04-20]
# update_timestamp:[2023-05-03T22\:00\:00.000Z TO 2023-05-04T12\:00\:00.000Z]

results = requests.get("https://api.adsabs.harvard.edu/v1/search/bigquery?{}".format(encoded_query), \
                       headers={'Authorization': 'Bearer ' + ADS_DEV_KEY})

print(results.json())

{'responseHeader': {'status': 0, 'QTime': 16, 'params': {'q': 'metadata_mtime:[2023-05-22T00\\:00\\:00.000Z TO 2023-05-23T00\\:00\\:00.000Z]', 'fl': 'bibcode,title,identifier', 'start': '0', 'internal_logging_params': 'X-Amzn-Trace-Id=Root=1-64777c7d-4e6682e33b0b713d59b0414f', 'fq': 'database:astronomy', 'rows': '2000', 'wt': 'json'}}, 'response': {'numFound': 339, 'start': 0, 'docs': [{'bibcode': '2022zndo...6788444T', 'identifier': ['2022zndo...6788444T', '10.5281/zenodo.6788444'], 'title': ['AMReX-Codes/amrex: AMReX 22.07']}, {'bibcode': '2023zndo...7790123T', 'identifier': ['2023zndo...7790123T', '10.5281/zenodo.7790123'], 'title': ['AMReX-Codes/amrex: AMReX 23.04']}, {'bibcode': '2019zndo...3379043E', 'identifier': ['2019zndo...3379043E', '10.5281/zenodo.3379043'], 'title': ['CMAQ']}, {'bibcode': '2018zndo...1412743B', 'identifier': ['2018zndo...1412743B', '10.5281/zenodo.1412743'], 'title': ['Keplergo/Lightkurve: V1.0B14']}, {'bibcode': '2015zndo...1079909R', 'identifier': ['2015

In [18]:
encoded_query = urlencode({"q": f"entdate:[2023-03-01 TO 2023-03-02]", 
                           "fl": "bibcode, identifier, title", 
                           "fq": "database:astronomy, property:article, property:(refereed OR eprint_openaccess)", 
                           "rows": 9999})

results = requests.get("https://api.adsabs.harvard.edu/v1/search/bigquery?{}".format(encoded_query), \
                       headers={'Authorization': 'Bearer ' + ADS_DEV_KEY})

ads_bibs = [paper['bibcode'] for paper in results.json()["response"]["docs"]]
# results.json()
print(len(ads_bibs))

{'responseHeader': {'status': 0, 'QTime': 284, 'params': {'q': 'entdate:[2023-03-01 TO 2023-03-02]', 'fl': 'bibcode,identifier,title', 'start': '0', 'internal_logging_params': 'X-Amzn-Trace-Id=Root=1-64777c1f-2ad4d88a04265cbf2875ea0a', 'fq': 'database:astronomy, property:article, property:(refereed OR eprint_openaccess)', 'rows': '2000', 'wt': 'json'}}, 'response': {'numFound': 44, 'start': 0, 'docs': [{'bibcode': '2023arXiv230300306B', 'identifier': ['arXiv:2303.00306', '10.48550/arXiv.2303.00306', '2023arXiv230300306B'], 'title': ['A massive interacting galaxy 525 million years after the Big Bang']}, {'bibcode': '2023arXiv230300410S', 'identifier': ['2023arXiv230300410S', '10.48550/arXiv.2303.00410', 'arXiv:2303.00410'], 'title': ['JWST/NIRSpec Measurements of the Relationships Between Nebular Emission-line Ratios and Stellar Mass at z~3-6']}, {'bibcode': '2023arXiv230300453E', 'identifier': ['2023arXiv230300453E', 'arXiv:2303.00453', '10.48550/arXiv.2303.00453'], 'title': ['Advances

In [25]:
headers = {
    'Authorization': f'Bearer {ADS_DEV_KEY}',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36',
    }

session = requests.Session()
retry = Retry(connect=3, backoff_factor=10)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

encoded_query = urlencode({"q": "bibcode:1998AJ....116.1897M", "fl": "title"})
results = session.get("https://api.adsabs.harvard.edu/v1/search/query?{}".format(encoded_query), headers=headers)

results.json()["response"]

{'numFound': 1,
 'start': 0,
 'docs': [{'title': ['Masers in Massive Star-Forming Regions Associated with the Brightest Steep-Spectrum IRAS Point Sources']}]}

## arXiv

In [9]:
start_day = "01"
start_month = "02"
start_year = "2023"

end_day = "02"
end_month = "02"
end_year = "2023"

start =  f"{start_year}{start_month}{start_day}" #start_date.strftime("%Y%m%d")
end = f"{end_year}{end_month}{end_day}" #end_date.strftime("%Y%m%d")
range_str = f"[{start}000000+TO+{end}000000]"
range_query = f"lastUpdatedDate:{range_str}"
# range_query = f"submittedDate:{range_str}"

base_url = "http://export.arxiv.org/api/query?"
full_query = f"search_query=%28astro-ph.GA+OR+astro-ph.CO+OR+astro-ph.EP+OR+astro-ph.HE+OR+astro-ph.IM+OR+astro-ph.SR%29+AND+{range_query}&max_results={9999}"
url = base_url + full_query

response = urlopen(url).read()
feed = feedparser.parse(response)
# print(feed)

api_arxiv_titles = [e.title_detail["value"] for e in feed.entries]
api_arxiv_ids = [e.id.split("/")[-1] for e in feed.entries]
api_arxiv_ids = [re.sub("v\d", "", id_) for id_ in api_arxiv_ids]

print(len(api_arxiv_titles))

{'bozo': False, 'entries': [{'id': 'http://arxiv.org/abs/2302.01293v1', 'guidislink': True, 'link': 'http://arxiv.org/abs/2302.01293v1', 'updated': '2023-02-01T02:09:03Z', 'updated_parsed': time.struct_time(tm_year=2023, tm_mon=2, tm_mday=1, tm_hour=2, tm_min=9, tm_sec=3, tm_wday=2, tm_yday=32, tm_isdst=0), 'published': '2023-02-01T02:09:03Z', 'published_parsed': time.struct_time(tm_year=2023, tm_mon=2, tm_mday=1, tm_hour=2, tm_min=9, tm_sec=3, tm_wday=2, tm_yday=32, tm_isdst=0), 'title': 'QLP Data Release Notes 003: GPU-based Transit Search', 'title_detail': {'type': 'text/plain', 'language': None, 'base': '', 'value': 'QLP Data Release Notes 003: GPU-based Transit Search'}, 'summary': 'The Quick-Look Pipeline (QLP; Huang et al. 2020, Kunimoto et al. 2021 and\nreferences therein) searches for transit signals in the multi-sector light\ncurves of several hundreds of thousand stars observed by TESS every 27.4-day\nsector. The computational expense of the planet search has grown considera

In [5]:
range_query

'lastUpdatedDate:[202321000000+TO+2023228000000]'

In [28]:
api_arxiv_ids

['2301.05335',
 '2304.09903',
 '2304.09905',
 '2304.09192',
 '2304.07446',
 '2304.09210',
 '2304.09817',
 '2304.09924',
 '2211.13576',
 '2304.09209',
 '2304.09890',
 '2304.09311',
 '2304.09794',
 '2304.08824',
 '2304.09368',
 '2205.07929',
 '2304.09140',
 '2206.10169',
 '2209.11078',
 '2304.09142',
 '2304.09474',
 '2304.09508',
 '2304.09204',
 '2304.09882',
 '2301.01312',
 '2304.08827',
 '2304.09350',
 '2210.08010',
 '2304.08839',
 '2304.07864',
 '2304.07962',
 '2304.08668',
 '2304.05924',
 '2304.09549',
 '2304.09977',
 '2304.09896',
 '2304.08898',
 '2304.09450',
 '2304.09878',
 '2205.07892',
 '2210.11622',
 '2304.09060',
 '2304.09396',
 '2304.09435',
 '2304.09816',
 '2304.09968',
 '2304.09220',
 '2304.09786',
 '2304.09942',
 '2304.09472',
 '2304.09129',
 '2211.00205',
 '2304.09222',
 '2304.09888',
 '2304.09894',
 '2303.16712',
 '2212.13454',
 '2304.09713',
 '2212.00839',
 '2304.09829',
 '2209.07693',
 '2204.06020',
 '2304.08995',
 '2304.09529',
 '2304.09539',
 '2304.08690',
 '2304.091

In [62]:
for e in feed.entries:
    print(e.title)
    break

HST Low Resolution Stellar Library


In [ ]:
n = 0
for title in api_arxiv_titles:
    print(title)
    n += 1
        
print(n)

In [11]:
directory = ".//data//arxiv emails//042023//"

with open(f"{directory}email.txt", 'r') as f:
    file_content = f.read()
    
email_arxiv_ids = re.findall(r'arXiv:\d{4}\.\d{5}', file_content) # looks like 'arXiv:2303.11347'
print(len(email_arxiv_ids))

email_arxiv_ids = [s[6:] for s in email_arxiv_ids]

Checking 88 papers


In [17]:
n = 0
for id_ in email_arxiv_ids:
    if id_ not in api_arxiv_ids:
        print(id_)
        n += 1
print(n)

1609.06324
2301.06432
2304.07364
2102.12495
4


In [18]:
n = 0
for id_, title in zip(api_arxiv_ids, api_arxiv_titles):
    if id_ not in email_arxiv_ids:
        print(id_, title)
        n += 1
print(n)

2304.07446 Evidence for Misalignment Between Debris Disks and Their Host Stars
2304.08824 A Dark Matter Probe in Accreting Pulsar-Black Hole Binaries
2205.07929 Dusty, Self-obscured Transients from Stellar Coalescence
2304.09140 A Sun-like star orbiting a boson star
2304.09142 Cosmology with Galaxy Cluster Properties using Machine Learning
2304.08827 The evolutionary route to form planetary nebulae with central neutron
  star - white dwarf binaries
2304.08839 The progenitors of Type Ia supernovae with asymptotic giant branch
  donors
2304.07864 Radio AGN Selection and Characterization in Three Deep-Drilling Fields
  of the Vera C. Rubin Observatory Legacy Survey of Space and Time
2304.08668 Emerging planetary nebulae within 3D spiral patterns
2304.08898 Galaxy pairs in The Three Hundred simulations II: studying bound ones
  and identifying them via machine learning
2304.06529 Cosmological application of the lens-redshift probability distribution
  with improved galaxy-scale gravitation